In [0]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub


In [0]:
# Check for GPU  availability
print('GPU', 'available' if tf.config.list_physical_devices('GPU') else 'not available')

GPU available


# Creating label.csv (merging the val and train data with the corresponding target)

## Creating the data frame serially

In [0]:
# Path's for data
path_trainNormal = '/content/drive/My Drive/Pnemonia classification/chest_xray/train/NORMAL'
path_trainPnemonia = '/content/drive/My Drive/Pnemonia classification/chest_xray/train/PNEUMONIA'
path_valNormal = '/content/drive/My Drive/Pnemonia classification/chest_xray/val/NORMAL'
path_valPnemonia = '/content/drive/My Drive/Pnemonia classification/chest_xray/val/PNEUMONIA'

In [0]:
# Getting the file form the directory.
id_label = [name for name in os.listdir(path_trainNormal)]
# Filtering the file with extenstion .jpeg
id_label = [ids for ids in id_label if ids.endswith(".jpeg")]

In [0]:
#Changing the list into the np array for ease.
id_label = np.array(id_label)

In [0]:
# Creating the data frame with pandas.
idDataframe = pd.DataFrame(id_label, columns=['id'])

In [0]:
# Viewing the data frame
idDataframe.head()

,id
0,NORMAL2-IM-0780-0001.jpeg
1,NORMAL2-IM-0804-0001.jpeg
2,NORMAL2-IM-0790-0001.jpeg
3,NORMAL2-IM-0803-0001.jpeg
4,NORMAL2-IM-0793-0001.jpeg


In [0]:
# As this is the Normal x-ray dessignated as 0
# New column with target 0
idDataframe['target'] = 0

In [0]:
# viewing the data
idDataframe.head()

,id,target
0,NORMAL2-IM-0780-0001.jpeg,0
1,NORMAL2-IM-0804-0001.jpeg,0
2,NORMAL2-IM-0790-0001.jpeg,0
3,NORMAL2-IM-0803-0001.jpeg,0
4,NORMAL2-IM-0793-0001.jpeg,0


Create a function that will convert the normal and pnemonia file into corresponding dataframe with the target value.
Algorithm:
1. Get the file name with the directory(path).
2. Filter the data with .jpeg extension.
3. Change the data into np.array for ease.
4. Create the dataframe for the data.
5. If normal if false then it is pnemonia data so target will be 1. 
6. Create a new column with target with correspoding value as per the normal or not.
7. Return the data frame.


In [0]:
# Creating the function for the creation of dataframe
def createDataFrame(path, normal=False):
  """
  Create a data frame and return it. 
  """
  # Get the name wit the directory.
  files = [file for file in os.listdir(path)]

  # Filter the data with the extension.
  files = [file for file in files if file.endswith(".jpeg")]

  # Creating the np array for ease.
  files = np.array(files)

  # Create a data frame 
  filesDataframe = pd.DataFrame(files, columns=['id'])

  # If normal is false then it is pnemonia data so target will be 1 else 0
  if 'target' in filesDataframe.columns:
    print('Column already exist!')
    return 0
  if normal:
    filesDataframe['target'] = 0
  else:
    filesDataframe['target'] = 1
  # Return the dataframe 
  return filesDataframe

In [0]:
# For path_trainPnemonia
dfTrainPnemonia = createDataFrame(path_trainPnemonia, normal=False)
# For path_trainNormal
dfTrainNormal = createDataFrame(path_trainNormal, normal=True)

# For path_valPnemonia
dfValPnemonia= createDataFrame(path_valPnemonia, normal=False)
# For path_valNormal
dfValNormal= createDataFrame(path_valNormal, normal=True)


In [0]:
dfValPnemonia.shape, dfValNormal.shape, dfTrainNormal.shape, dfTrainPnemonia.shape

((8, 2), (8, 2), (1341, 2), (3875, 2))

In [0]:
dfDict = {'valPnemonia':dfValPnemonia,
          'valNormal': dfValNormal,
          'trainNormal': dfTrainNormal,
          'trainPnemonia': dfTrainPnemonia}

Creating a function that will concat the dataframe horizontally and vertically (This function will take dictonary of the data frame)
Algorithm:
1. Reset the index just left one. (df.reset_index(drop=True))
2. Create a variable for the first value in dict ie df1.
3. Loop throught the dict.items() 
  1. If i == 1 then continue.
  2. New variable resetdf whose index is reset using(values.reset_index  (drop=True))
  3. Concat the first values df1 and the resetdf.
  4. Now the first data frame df1 will be the new dataframe df.
2. For vertical concat axis = 0 ie  (pd.concat([df1, df2], axis=0)
3. For horizontal concat axis = 1 ie column (pd.concat([df1, df2], axis=1)


In [0]:
# Creating a function for the concat of dataframe
def dfConcat(dfDict):
  """
  This is for concat of df vertical.
  """
  # Reset the index just left one.
  df1 = list(dfDict.values())[0] 
  for i, (keys, values) in enumerate(dfDict.items()):
    if i is 0:
      continue
    resetdf = values.reset_index(drop=True)
    df = pd.concat([df1, resetdf], axis = 0)
    df1 = df
  return df1

In [0]:
dfx_rayClassification = dfConcat(dfDict)
dfx_rayClassification

,id,target
0,person1946_bacteria_4874.jpeg,1
1,person1947_bacteria_4876.jpeg,1
2,person1950_bacteria_4881.jpeg,1
3,person1949_bacteria_4880.jpeg,1
4,person1946_bacteria_4875.jpeg,1
...,...,...
3870,person1107_bacteria_3048.jpeg,1
3871,person1106_virus_1829.jpeg,1
3872,person110_virus_205.jpeg,1
3873,person1108_bacteria_3049.jpeg,1


In [0]:
dfx_rayClassification.head()

,id,target
0,person1946_bacteria_4874.jpeg,1
1,person1947_bacteria_4876.jpeg,1
2,person1950_bacteria_4881.jpeg,1
3,person1949_bacteria_4880.jpeg,1
4,person1946_bacteria_4875.jpeg,1


In [0]:
# shuffle the sample and reset the index 
dfx_rayClassification = dfx_rayClassification.sample(frac=1).reset_index(drop=True)

In [0]:
# Saving the data frame into csv file.
dfx_rayClassification.to_csv('/content/drive/My Drive/Pnemonia classification/label.csv', index=False)

In [0]:
df = pd.read_csv('/content/drive/My Drive/Pnemonia classification/label.csv')